<a href="https://colab.research.google.com/github/julyml/ep_ia/blob/main/multilayer_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

class multilayer_perceptron():
    def __init__(self,tamanho_escondida,tamanho_saida,taxa_aprendizado, ativacao,d_ativacao, bias_entrada, bias_escondida,tamanho_entrada):
        # carrega o arquivo de dados de uma url
        self.ativacao = ativacao
        self.d_ativacao = d_ativacao
        #caracterısticas extraıdas (n) de uma imagem 
        self.n = tamanho_entrada
        # print("Caracteristicas n:\n",self.tamanho_entrada)

        #numero de neuronios na camada escondida
        self.p = tamanho_escondida  
        # print("Tamanho da camada escondida p\n",self.p)

        # numero de pesos da camada escondida é p.(n+ 1)
        self.n_v = self.p *(self.n + 1)
        # print("Número de pesos da camada escondida:\n",self.n_v) 
        
        #pesos iniciais da camada escondida
        self.V = self.init_weights_hidden()
        self.V.to_csv('Pesos iniciais da camanda escondida.csv')
        # print("Pesos iniciais da camada escondida - V\n",self.V)   

        #numero de neuronios na camada de saida
        self.m = tamanho_saida 
        # print("Tamanho da camada de saída\n",self.m)
        
        #numero de pesos na camada de saida m.(p + 1)
        self.n_w = self.m * (self.p + 1)
        # print("Numero de pesos da camada de saída\n",self.n_w)

        #pesos iniciais da camada de saida
        self.W = self.init_weights_output() 
        self.W.to_csv('Pesos iniciais da camanda de saida.csv')
        # print("pesos iniciais da camada de saida - W\n ", self.W)

        # define um numero entre 0 e 1 como taxa de aprendizado
        self.α = taxa_aprendizado
        # print("Taxa de aprendizado\n",self.α)

        # define o valor do bias na camada de entrada
        self.bias_entrada = bias_entrada
        # print("Bias da camada de entrada\n", self.bias_entrada)

        # define o valor do bias na camada escondida
        self.bias_escondida = bias_escondida
        self.bias_escondida =  pd.DataFrame([bias_escondida])
        # print("Bias da camada escondida\n", self.bias_escondida)

        parametros = [
                      ['Caracteristicas extraídas',self.n],
                      ['Tamanho da camada escondida',self.p],
                      ['Número de pesos da camada escondida',self.n_v],
                      ['Tamanho da camada de saída',self.m],
                      ['Numero de pesos da camada de saída',self.n_w],
                      ['Taxa de aprendizado',self.α],
                      ['Bias da camada de entrada',self.bias_entrada],
                      ['Bias da camada escondida',self.bias_escondida]]    

        pd.DataFrame(data=parametros, columns=['parametro','valor']).to_html('parametros_iniciais.html')

    def init_weights_hidden(self): 
      #retorna uma matriz com pesos aleatorios para camada escondida
      return pd.DataFrame(np.random.rand(self.p,(self.n+1)))

    def init_weights_output(self): 
      #inicializa os pesos da camada de saida
      return pd.DataFrame(np.random.rand(self.m,(self.p+1))) 

    def feedforward(self,  row): 
      '''
      Passos 3, 4 e 5 do algoritmo da pagina 294 do livro da Faussett
      '''
      #row ja vem com o bias 
      row = pd.DataFrame(row)     
      # print("X:\n",row)      
      # print('V:\n',self.V)
      # print('W:\n',self.W)

      Z_in = self.V.dot(row) 
      Z_in.columns = range(Z_in.shape[1])
      # print("Z_in:\n",Z_in)

      #aplica a funcao de ativação sobre o dados da camada escondida
      Z = Z_in.applymap(self.ativacao)
      Z.columns = range(Z.shape[1])
      # print("Z:\n",Z)

      # adicionamos o bias da camada escondida  = z chapeu invertido
      Z_bias_escondida =  pd.concat([self.bias_escondida, Z]).reset_index(drop = True)
      Z_bias_escondida.columns = range(Z_bias_escondida.shape[1])
      # print('Z_bias_escondida\n',Z_bias_escondida)      

      # calculamos o valor de Y inativado
      Y_in = self.W.dot(Z_bias_escondida)
      Y_in.columns = range(Y_in.shape[1])
      # print('Y_in:\n',Y_in)

      # #aplica a funcao de ativação sobre o dados da camada de saida (Y_in)
      Y = Y_in.apply(self.ativacao)
      Y.columns = range(Y.shape[1])
      # print('Y:\n',Y)
      return Z_in,Z_bias_escondida,Z,Y_in,Y

    def backpropagation(self,Z_in,Z_bias_escondida,Z,Y_in,Y,tk,xk):
      # Camada de saida          
          # calculo a taxa de erro ou seja o esperado menos o recebido
        # aplica a sigmoide derivada ao Y_in
        Y_D = Y_in.apply(self.d_ativacao) 
        # print('Y_D:\n',Y_D)

        # calculo da informaçao de erro
        δs = (tk-Y) * Y_D
        # print('δs\n',δs)

        # calculo do termo de correçao
        Δw = list()
        for i, δsi in δs.iterrows():
          Δw.append((self.α * δsi * Z_bias_escondida).T)
        Δw = pd.concat(Δw,ignore_index=True)
        # print('Δw\n',Δw)
        
        '''
          Passo 7 do algoritmo descrito na página 294 do livro da Fausset
        '''

        # Camada escondida HIDDEN
          # calculo a taxa de erro ou seja o esperado menos o recebido          
        # aplica a sigmoide derivada ao Z_in          
        Z_D = Z_in.apply(self.d_ativacao)  
        # print('Z_D:\n',Z_D)

        #calculo da informaçao de erro
        δh = pd.DataFrame()
        w_without_first_Columns = self.W.iloc[:, 1:] # mas por que eu ignorei a primeira coluna
        w_without_first_Columns.columns = range(w_without_first_Columns.shape[1])

        for wii in w_without_first_Columns:
          w_temp = pd.DataFrame(w_without_first_Columns[wii])
          w_temp.columns = range(w_temp.shape[1])
          δhi =  Z_D.iloc[wii] * (δs * w_temp).sum()
          δh = δh.append([δhi],ignore_index=True)          
        # print('δh:\n',δh)

        # calculo do termo de correçao
        Δv_lista = list()
        for i, δhi in δh.iterrows():
          xk = pd.DataFrame(xk)    
          xk.columns = range(xk.shape[1])
          Δv_lista.append((self.α * δhi * xk).T)
        Δv = pd.concat(Δv_lista,ignore_index=True)
        # print('Δv\n',Δv)
       
        return δs,Δw,δh,Δv
    
    def train(self, dados,threshold = 1e-3):
      
      targets = dados.iloc[:,self.n:]
      targets.columns = range(targets.shape[1])

      inputs = dados.iloc[: , :self.n]
      inputs.insert(inputs.shape[0]+1, "bias", self.bias_entrada)
      inputs.columns = range(inputs.shape[1])        

      square_error = 2 * threshold
      counter = 1
      δ_list_ = []
      while square_error > threshold:        
        square_error = 0
        for (i, input), (j, target)  in zip(inputs.iterrows(),targets.iterrows()):
          # entrada do treino = X
          xk = input.reset_index(drop=True) 
          # saida esperada para a entrada acima = Target 
          tk = pd.DataFrame(target).reset_index(drop=True) 
          tk.columns = range(tk.shape[1])

          # Passo os dados de input (Xk) pelo passo feed
          Z_in,Z_bias_escondida,Z,Y_in,Y = self.feedforward(xk) 

          # calculo a taxa de erro usando o Y
          error = tk.reset_index(drop=True)-Y
          square_error = (square_error + error.pow(2).sum())[0]
          # square_error = square_error + sum(error*2)
          '''
           Passo 6 do algoritmo descrito na página 294 do livro da Fausset
          '''
          δs,Δw,δh,Δv = self.backpropagation(Z_in,Z_bias_escondida,Z,Y_in,Y,tk,xk)   

          δ_list_.append([counter,δh,δs])

          '''
           Passo 8 do algoritmo descrito na página 294 do livro da Fausset
          '''
          #mudar os pesos das camadas
          self.W =  self.W.add(Δw, fill_value=0)
          self.V = self.V.add(Δv, fill_value=0)

        square_error = square_error / len(dados.index)
        # print('Erro médio quadrado:\n',square_error)
        counter+=1      

      self.V.to_csv('Pesos finais da camanda de saida.csv')
      self.W.to_csv('Pesos finais da camanda de saida.csv')
      header = ['iteracao','erros escondida', 'erros saida']
      pd.DataFrame(δ_list_, columns=header).to_csv(f'Erros das camandas.csv')
      # print('Quantidade de iterações do treino:\n',counter)
      return 'Rede treinada com sucesso'

    def predict(self, xk):
      # adiciono o bias a entrada de teste
      xk = xk.append(pd.Series([self.bias_entrada]),ignore_index=True)
      _,_,_,_,Y = self.feedforward(xk)
      return Y

    def test_dataset(self, dados):
      targets = dados.iloc[:,self.n:]
      targets.columns = range(targets.shape[1])

      inputs = dados.iloc[: , :self.n]
      inputs.columns = range(inputs.shape[1])   

      outputs = list()

      for (i, input), (j, target)  in zip(inputs.iterrows(),targets.iterrows()):        
        input = input.reset_index(drop=True)
        tk = pd.DataFrame(target).reset_index(drop=True)
        tk.columns = range(tk.shape[1])
        result = self.predict(input)[0].to_list()
        outputs.append([result,target[0].to_list() ])
      
      return outputs

    def matriz_confusao_caracteres(self,outputs):
      # output[0]: lista de lista com todos os outputs reais de teste
      # output[1]: lista de lista de todos os targets de teste    

      lista_letras = ['A','B','C','D','E','F','G']
      target_list = ['[1,0,0,0,0,0,0]','[0,1,0,0,0,0,0]','[0,0,1,0,0,0,0]','[0,0,0,1,0,0,0]','[0,0,0,0,1,0,0]','[0,0,0,0,0,1,0]','[0,0,0,0,0,0,1]']

      dicionario_letras = dict(zip(target_list,lista_letras))
      
      #cria uma matriz do tamanho do caracteres para criar a matriz de confusão multiclasse
      matrix = pd.DataFrame(0, index=lista_letras, columns=lista_letras)

      for output in outputs:
        out = [int(round(x,1)) for x in output[0]]
        chave_linha = str(out).replace(' ', '')
        chave_coluna = str(output[1]).replace(' ', '')
        linha = dicionario_letras[chave_linha]
        coluna = dicionario_letras[chave_coluna]      
        matrix.at[linha,coluna]=matrix.at[linha,coluna]+1
      return matrix

    def generate_matriz_confusao(self,dados):
      outputs = self.test_dataset(dados)
      return self.matriz_confusao_caracteres(outputs)

In [ ]:
def sigmoide(n):
  #Funcao de ativaçao sigmoide
  return  1.0 / (1.0 + np.exp(-n))

def derivada_sigmoide(n):       
  #Funcao derivada da sigmoide
  return sigmoide(n) * (1 - sigmoide(n))  


mlp = multilayer_perceptron(tamanho_escondida=15,
                            tamanho_saida=7,
                            taxa_aprendizado=0.5,                            
                            ativacao = sigmoide,
                            d_ativacao = derivada_sigmoide,
                            bias_entrada=1,
                            bias_escondida=1,
                            tamanho_entrada=63)

In [ ]:
dataset_url = 'https://raw.githubusercontent.com/julyml/ep_ia/main/Dados/caracteres-limpo.csv'

dados = pd.read_csv(dataset_url, header=None,delimiter=',') 

msk = np.random.rand(len(dados)) < 0.8

treino = dados[msk]

teste = dados[~msk]

In [ ]:
mlp.train(treino)

'Rede treinada com sucesso'

In [ ]:
mlp.predict(pd.Series([-1,-1,1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,1,1,1,1,1,-1,-1,1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,1,-1,1,1,1,-1,1,1,1]).T)

In [ ]:
mlp.generate_matriz_confusao(teste)